In [1]:
#!/usr/bin/env python3

import decimal
import sys
import gurobipy as g
import numpy as np
from itertools import permutations, combinations

In [2]:
file_path = 'instances/example-1.txt'

In [3]:
f = open(file_path)

data = []

for line in f:
    data_line = []
    for j in line.split(' '):
        data_line.append(j)
    data.append(data_line)

# for i in data:
#     for j in i:
#         print(j, end=" ")


In [4]:
n = int(data[0][0])
w = int(data[0][1])
h = int(data[0][2])

picture = []
piece = []

for i in range(0, n):
    piece = np.asarray(data[i+1])
    piece = np.resize(piece, (h, w*3))
    picture.append(piece)

picture = np.array(picture)
picture = picture.astype(int)

In [5]:
def distance(A, B):
    A = np.array(A[:, -3:])
    B = np.array(B[:, :3])
    return np.sum(np.absolute(A - B))

def compute_distance(A, B):
    height = A.shape[0]
    width = A.shape[1]
    A = np.asarray(A)
    B = np.asarray(B)
    A = A[:, width - 3: width]
    B = B[:, 0:3]
    return np.sum(np.absolute(A - B))

# dist = np.zeros((n+1, n+1), int)
# dist = {}

# print(dist[0][1])

# perm = permutations(np.arange(n), 2)

# for p in perm:
#     # key = str(p[0])+"-"+str(p[1])
#     dist[p[0], p[1]] = distance(picture[p[0]], picture[p[1]])

# for p in range(n+1):
    # dist[0, str(p)] = 0
    # dist[str(p), 0] = 0

# print(dist)


In [6]:
stripes = []
stripe = []
for i in range(0, n):
    stripe = np.asarray(data[i + 1])
    stripe = np.resize(stripe, (h, 3*w))
    stripes.append(stripe)

stripes = np.asarray(stripes)

print(stripes)
print(picture)

[[['255' '0' '0' '0' '0' '255']
  ['0' '128' '128' '0' '255' '0']
  ['0' '0' '255' '255' '0' '0']
  ['0' '255' '0' '255' '0' '0\n']]

 [['0' '0' '255' '0' '255' '0']
  ['0' '0' '255' '0' '255' '0']
  ['255' '0' '0' '0' '0' '255']
  ['0' '255' '0' '0' '255' '0\n']]

 [['0' '0' '255' '0' '0' '255']
  ['0' '0' '255' '0' '0' '255']
  ['0' '0' '255' '0' '0' '255']
  ['0' '0' '255' '0' '0' '255']]]
[[[255   0   0   0   0 255]
  [  0 128 128   0 255   0]
  [  0   0 255 255   0   0]
  [  0 255   0 255   0   0]]

 [[  0   0 255   0 255   0]
  [  0   0 255   0 255   0]
  [255   0   0   0   0 255]
  [  0 255   0   0 255   0]]

 [[  0   0 255   0   0 255]
  [  0   0 255   0   0 255]
  [  0   0 255   0   0 255]
  [  0   0 255   0   0 255]]]


In [7]:
m = g.Model()

vars = {}

for i in range(n):
    for j in range(n):
        if i != j:
            dist = distance(picture[i], picture[j])
            vars[i, j] = m.addVar(obj=dist, vtype=g.GRB.BINARY, name='e' + str(i) + '_' + str(j))
        else:
            vars[i, j] = m.addVar(obj=0, ub=0, vtype=g.GRB.BINARY, name='e' + str(i) + '_' + str(j))
    m.update()

for k in range(n):
    vars[n, k] = m.addVar(obj=0, vtype=g.GRB.BINARY, name='e' + str(n) + '_' + str(k))

for k in range(n):
    vars[k, n] = m.addVar(obj=0, vtype=g.GRB.BINARY, name='e' + str(k) + '_' + str(n))

vars[n, n] = m.addVar(obj=0, ub=0, vtype=g.GRB.BINARY, name='e' + str(n) + '_' + str(n))
m.update()


for j in range(n + 1):
    m.addConstr(g.quicksum(vars[i, j] for i in range(n + 1)) == 1)

for i in range(n + 1):
    m.addConstr(g.quicksum(vars[i, j] for j in range(n + 1)) == 1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-12


In [8]:
def find_one_tour(edges):
    tour = []
    tour.append(edges[0])
    start = edges[0][0]
    temp = edges[0][1]
    edges.remove(tour[0])

    while True:
        for edge in edges:
            if edge[0] == temp:
                tour.append(edge)
                if edge[1] != start:
                    temp = edge[1]
                    edges.remove(edge)
                    break
                else:
                    edges.remove(edge)
                    return tour, edges


def find_shortest_tour(edges):
    shortest_tour = []
    while len(edges) > 1:
        tour, edges = find_one_tour(edges)
        if len(shortest_tour) == 0:
            shortest_tour = tour
        else:
            if len(tour) < len(shortest_tour):
                shortest_tour = tour
    return shortest_tour

def subtourelim(model, where):
  if where == g.GRB.callback.MIPSOL:
    selected = []
    # make a list of edges selected in the solution
    for i in range(n+1):
      sol = model.cbGetSolution([model._vars[i,j] for j in range(n+1)])
      selected += [(i,j) for j in range(n+1) if sol[j] > 0.5]
    # find the shortest cycle in the selected edge list
    tour = find_shortest_tour(selected)
    if len(tour) < n:
      # add a subtour elimination constraint
      expr = 0
      for i in range(len(tour)):
          expr += model._vars[tour[i][0], tour[i][1]]
      model.cbLazy(expr <= len(tour)-1)

In [9]:
# m = g.Model()
# capitals = [0, 1, 2]

# # Variables: is city 'i' adjacent to city 'j' on the tour?
# vars = m.addVars(dist.keys(), obj=dist, vtype=g.GRB.BINARY, name='x')

# # Symmetric direction: Copy the object
# for i, j in vars.keys():
#     vars[j, i] = vars[i, j]  # edge in opposite direction

# # Constraints: two edges incident to each city
# cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

# print(cons)
# print(capitals)

In [10]:

# def subtourelim(model, where):
#     if where == g.GRB.Callback.MIPSOL:
#         # make a list of edges selected in the solution
#         vals = model.cbGetSolution(model._vars)
#         selected = g.tuplelist((i, j) for i, j in model._vars.keys()
#                              if vals[i, j] > 0.5)
#         # find the shortest cycle in the selected edge list
#         tour = subtour(selected)
#         if len(tour) < len(capitals):
#             # add subtour elimination constr. for every pair of cities in subtour
#             model.cbLazy(g.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
#                          <= len(tour)-1)

# # Given a tuplelist of edges, find the shortest subtour

# def subtour(edges):
#     unvisited = capitals[:]
#     cycle = capitals[:] # Dummy - guaranteed to be replaced
#     while unvisited:  # true if list is non-empty
#         thiscycle = []
#         neighbors = unvisited
#         while neighbors:
#             current = neighbors[0]
#             thiscycle.append(current)
#             unvisited.remove(current)
#             neighbors = [j for i, j in edges.select(current, '*')
#                          if j in unvisited]
#         if len(thiscycle) <= len(cycle):
#             cycle = thiscycle # New shortest subtour
#     return cycle

In [11]:
m._vars = vars
m.params.LazyConstraints = 1
m.optimize(subtourelim)
m.write('model.lp')

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x3f97cddb
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2550.0000000
Presolve removed 0 rows and 4 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 1.785000e+03, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1785.0000000 1785.00000  0.00%     -    0s

Expl

In [12]:
output_file = open('output.txt', 'w+')

final_tour = []
for key, value in vars.items():
    if value.X == 1:
        final_tour.append(key)

final_tour = find_shortest_tour(final_tour)
result = []

for i in range(n+1):
    if final_tour[i][0] == n:
        dummy_index = i


result.append(final_tour[i][0])
part1 = result[dummy_index+1: len(final_tour)]
part2 = result[0:dummy_index]
result = np.concatenate((part1, part2), axis=0)
output_file.write(" ".join(map(str, result.astype(int))))

1

In [13]:
# # Retrieve solution

# vals = m.getAttr('x', vars)
# selected = g.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

# for i in selected:
#     print(selected)

# tour = subtour(selected)
# assert len(tour) == len(capitals)